# External Python Logic (Notebook)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LineageLogic/LakeLogic/blob/main/examples/06_advanced_workflows/external_python_logic/gold/sales_gold.ipynb) 
[![GitHub Repo](https://img.shields.io/badge/GitHub-Repo-blue?logo=github)](https://github.com/LineageLogic/LakeLogic/blob/main/examples/06_advanced_workflows/external_python_logic/gold/sales_gold.ipynb)

## Business Scenario

Some business logic is too complex for pure YAML. You need a way to extend contracts with notebook-based logic while keeping validation in place.

## Value Proposition

- Use notebooks for complex transformations
- Keep contracts as the source of truth
- Produce Gold outputs consistently

---

## Goals

1. Load staged data
2. Apply external notebook logic
3. Write Gold outputs


In [1]:
from pathlib import Path
import csv
from collections import defaultdict
from datetime import datetime, timezone


def build_sales_gold(params=None, input_path=None, output_path=None):
    if params is None:
        params = globals().get("LAKELOGIC_PARAMS", {}) or {}

    contract_dir = Path(params.get("lakelogic_contract_dir", Path.cwd()))
    default_input = contract_dir / "data" / "sales.csv"
    if not default_input.exists():
        candidate = Path.cwd() / "examples" / "06_advanced_workflows" / "external_python_logic"
        if (candidate / "data" / "sales.csv").exists():
            contract_dir = candidate
            default_input = contract_dir / "data" / "sales.csv"

    if input_path is None:
        input_path = params.get("lakelogic_input_path") or default_input
    if output_path is None:
        output_path = params.get("lakelogic_output_path") or (contract_dir / "output" / "gold_fact_sales.csv")

    input_path = Path(input_path)
    totals = defaultdict(float)

    if not input_path.exists():
        print(f"Missing input file: {input_path}")
        return []

    with input_path.open("r", newline="", encoding="utf-8") as handle:
        reader = csv.DictReader(handle)
        for row in reader:
            salesperson = row.get("salesperson_id")
            if not salesperson:
                continue
            try:
                totals[salesperson] += float(row.get("amount", 0) or 0)
            except (TypeError, ValueError):
                continue

    processed_at = datetime.now(timezone.utc).isoformat()
    rows = [
        {
            "salesperson_id": int(salesperson) if str(salesperson).isdigit() else salesperson,
            "total_amount": total,
            "processed_at": processed_at,
        }
        for salesperson, total in sorted(totals.items(), key=lambda item: str(item[0]))
    ]

    output_path = Path(output_path) if output_path else None
    if output_path:
        output_path.parent.mkdir(parents=True, exist_ok=True)
        with output_path.open("w", newline="", encoding="utf-8") as handle:
            writer = csv.DictWriter(handle, fieldnames=["salesperson_id", "total_amount", "processed_at"])
            writer.writeheader()
            writer.writerows(rows)

    return rows


rows = build_sales_gold()
rows[:5]


Missing input file: D:\Github\_SaaS\SaaS_remotevest\data\sales.csv


[]